In [6]:
import os
import glob
from PIL import Image
import math
from collections import Counter
from google.cloud import vision
import re
import cv2
import subprocess
import pandas as pd 
import numpy as np

# heic-jpg converter


In [2]:
from PIL import Image
from pillow_heif import register_heif_opener
import pillow_heif

In [3]:
help(register_heif_opener)
register_heif_opener()

Help on function register_heif_opener in module pillow_heif.as_plugin:

register_heif_opener(**kwargs) -> None
    Registers a Pillow plugin for HEIF format.

    :param kwargs: dictionary with values to set in options. See: :ref:`options`.



In [8]:
heic_dir= 'name'
output_dir= 'name'


# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# List of HEIC files
heic_files = [f for f in os.listdir(heic_dir) if f.lower().endswith('.heic')]

# Convert HEIC to JPG and save in the output directory
for photo in heic_files:
    heic_path = os.path.join(heic_dir, photo)
    
    # Replace file extension from .HEIC to .jpg
    jpg_photo = photo.replace('.HEIC', '.jpg').replace('.heic', '.jpg')
    
    # Define the output path for the new JPG file
    output_path = os.path.join(output_dir, jpg_photo)

    # Call heif-convert command
    try:
        subprocess.run(['heif-convert', heic_path, output_path], check=True)
        print(f"Converted {photo} to {jpg_photo}")
    except subprocess.CalledProcessError as e: 
        print(f"Error converting {photo}: {e}")

# Optionally, open and process the converted images using Pillow
for photo in os.listdir(output_dir):
    if photo.lower().endswith('.jpg'):
        jpg_path = os.path.join(output_dir, photo)
        try:
            with Image.open(jpg_path) as img:
                img.save(jpg_path, format='JPEG')
            print(f"Processed {photo} with Pillow")
        except Exception as e:
            print(f"Error processing {photo} with Pillow: {e}")

print("Conversion and processing completed.")




File contains 1 image
Written to name/Lidl_Crackers_3.jpg
Converted Lidl_Crackers_3.heic to Lidl_Crackers_3.jpg
File contains 1 image
Written to name/Lidl_veg_sausage_2.jpg
Converted Lidl_veg_sausage_2.heic to Lidl_veg_sausage_2.jpg
File contains 1 image
Written to name/Lidl_Crownfield_MuseliBar_WhippedMint.jpg
Converted Lidl_Crownfield_MuseliBar_WhippedMint.heic to Lidl_Crownfield_MuseliBar_WhippedMint.jpg
File contains 1 image
Written to name/Lidl_KingPrawn(Crustaceans).jpg
Converted Lidl_KingPrawn(Crustaceans).heic to Lidl_KingPrawn(Crustaceans).jpg
File contains 1 image
Written to name/Lidl_sliced_mushroom.jpg
Converted Lidl_sliced_mushroom.heic to Lidl_sliced_mushroom.jpg
File contains 1 image
Written to name/Lidl_Sliced_ChickenFillet_Potato.jpg
Converted Lidl_Sliced_ChickenFillet_Potato.heic to Lidl_Sliced_ChickenFillet_Potato.jpg
File contains 1 image
Written to name/Lidl_Sliced_Mature_RedCheddar.jpg
Converted Lidl_Sliced_Mature_RedCheddar.heic to Lidl_Sliced_Mature_RedCheddar.j

In [9]:
# Find all .heic files in the specified folder
heic_files = glob.glob(os.path.join(output_dir, '*.heic'))

# Loop through the list of files and delete each one
for file_path in heic_files:
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except Exception as e:
        print(f"Failed to delete {file_path}: {e}")

Deleted: name/Lidl_Crackers_3.heic
Deleted: name/Lidl_veg_sausage_2.heic
Deleted: name/Lidl_Crownfield_MuseliBar_WhippedMint.heic
Deleted: name/Lidl_KingPrawn(Crustaceans).heic
Deleted: name/Lidl_sliced_mushroom.heic
Deleted: name/Lidl_Sliced_ChickenFillet_Potato.heic
Deleted: name/Lidl_Sliced_Mature_RedCheddar.heic
Deleted: name/Lidl_Irish_Vintage_RedCheddar.heic
Deleted: name/Lidl_Cookie_biscuit_Dark.heic
Deleted: name/Lidl_HoneyNut_Clusters.heic
Deleted: name/Lidl_Fresh_Egg_Penne_Pasta.heic
Deleted: name/Lidl_Champagne_dessert.heic
Deleted: name/Lidl_Corn_Flakes.heic
Deleted: name/Lidl_MultiGrain_shapes.heic
Deleted: name/Lidk_Frozen_Fish_Fillets.heic
Deleted: name/Lidl_TowerGate_Biscuit_Mix.heic
Deleted: name/Lidl_4_seasoned_Irish_beef_2.heic
Deleted: name/Lidl_Vegan_Sausage.heic
Deleted: name/Lidl_golden_bake_Ham&Cheese_2.heic
Deleted: name/Lidl_Alfredo_pizza_sub.heic
Deleted: name/Lidl_breaded_chicken_fillet.heic
Deleted: name/Lidl_HoneyNut_Clusters_MilkChocolate.heic
Deleted: na

# Resize images before OCR


In [10]:
def resize_image(input_path, output_path, size=(1024, 1024)):
    with Image.open(input_path) as img:

        # Resize the image
        img.thumbnail(size, Image.LANCZOS)
        
        # Save the resized image
        img.save(output_path, "JPEG")

def process_images(input_folder, output_folder, size=(1024, 1024)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.jpg'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            resize_image(input_path, output_path, size)
            print(f"Processed {filename}")

# Define input and output directories
input_folder = 'name'
output_folder = 'Preprocessed_2'

# Process all images
process_images(input_folder, output_folder)

Processed Lidl_TowerGate_Cookie_Cranberrry.jpg
Processed Lidl_PotatoWaffle_BirdsEye.jpg
Processed Lidl_chicken_nuggets_2.jpg
Processed Lidl_HoneyNut_Clusters_MilkChocolate.jpg
Processed Lidl_KingPrawn(Crustaceans).jpg
Processed Lidl_chicken_pop.jpg
Processed Lidl_Cooked_marinatedChickenBreast_Rice.jpg
Processed Lidl_Neo_biscuit_White.jpg
Processed Lidl_Sliced_ChickenFillet_Potato.jpg
Processed Lidl_Sliced_Irish_med_RedCheddar.jpg
Processed Lidl_Crownfield_bf_biscuit_strawberry.jpg
Processed Lidl_MultiGrain_Hoops_Chocolate.jpg
Processed Lidl_CocoaFood_Cookies_Vanilla.jpg
Processed Lidl_Wholewheat_FusiliPasta.jpg
Processed Lidl_Vegan_Sausage.jpg
Processed Lidl_Delux_Beef_Burger_patty.jpg
Processed Lidl_TowerGate_Biscuit_Mix.jpg
Processed Lidl_Deluxe_Tiramisu.jpg
Processed Lidl_chicken_pop_2.jpg
Processed Lidl_Irish_Vintage_RedCheddar.jpg
Processed Lidl_Alfredo_pizza_sub_2.jpg
Processed Lidl_Malt_Wheaties.jpg
Processed Lidl_Milbona_profiteroles_dessert.jpg
Processed Lidl_chicken_strips.jp

In [11]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='plasma-matter-429520-e4-37c272513bf7.json'
WORD = re.compile(r"\w+")

# API call


In [12]:
def detect_text(path):
    """Detects text in the file."""

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # for non-dense text 
    # response = client.text_detection(image=image)
    # for dense text
    response = client.document_text_detection(image=image)
    texts = response.text_annotations
    ocr_text = []

    for text in texts:
        ocr_text.append(f"\r\n{text.description}")

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return ocr_text

In [13]:
def detect_text(path):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()

    # Check if the file exists and is readable
    if not os.path.isfile(path):
        print(f"File not found: {path}")
        return []

    try:
        with open(path, "rb") as image_file:
            content = image_file.read()
    except Exception as e:
        print(f"Error reading file {path}: {e}")
        return []

    image = vision.Image(content=content)

    try:
        response = client.text_detection(image=image)
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return []

    if response.error.message:
        print(f"API error for image {path}: {response.error.message}")
        return []

    texts = response.text_annotations
    return texts


In [14]:
#Calling API, detecting text 
#saving cleaned text in dictionary 
output_folder = 'processed_images'
text={}

# Get the list of images and process the first 10 images
images = os.listdir(output_folder)

for img in images:
    image_path = os.path.join(output_folder, img)
    detected_text = detect_text(image_path)
    only_name = img.replace('.jpg', '')

    if detected_text:
        clean_text = detected_text[0].description
        clean_text = clean_text.replace('\r', '').replace('\n', ' ').strip()
        text[only_name] = clean_text.lower()
    else:
        print(f"Skipping image due to detection issues: {image_path}")

print(text)

{'shepardless-pie': 'nutritional information typical values per 100g energy 331kj | 79kcal & fat 1.7g 4 of which saturates 0.2g 0 carbohydrate 13g 3 of which sugars 2.1g 5 fibre 2.0g 5 protein 1.8g 4 salt 0.39g 0 this pack contains 4 servings.', 'Lidl_TowerGate_Cookie_Cranberrry': 'lidl nutrition information per cookie typical values per 100g energy 2050kj/ 25g %ri* 512kj/ 490kcal 122kcal 6% fat 22.9g 5.7g 8% of which saturates 10.9g 2.7g 14% carbohydrate 64.4g 16.1g 6% of which sugars 45.2g 11.3g 13% fibre 2.7g 0.7g protein 5.1g 1.3g 3% salt 0.67g 0.17a 3%', 'chocuer_milk_chocolate': 'nutrition this bar contains 6 servings. typical values per 100g per 5 squares (approx. 33g) energy 2371kj/569kcal 789kj/189kcal fat 36.4g 12.19 of which saturates 22.89 7.6g carbohydrate 51.6g 17.2g of which sugars 49.7g 16.6g fibre 1.99 0.6g protein 7.7g 2.6g salt 0.18g 0.06g', 'john-west-tuna-chunks-in-brine': 'nutritional info t per 100g per can (56g) a 451 kj/106 kcal 0.79 253 kj/59 kcal 0.4g s (0.2g

In [21]:
# Storing OCR-Text in dataframe with name
ocr_data=pd.DataFrame({"name": text.keys(), "text": text.values(),"Val":''})
ocr_data


,name,text,Val
0,shepardless-pie,nutritional information typical values per 100...,
1,Lidl_TowerGate_Cookie_Cranberrry,lidl nutrition information per cookie typical ...,
2,chocuer_milk_chocolate,nutrition this bar contains 6 servings. typica...,
3,john-west-tuna-chunks-in-brine,nutritional info t per 100g per can (56g) a 45...,
4,Lidl_PotatoWaffle_BirdsEye,nutrition information average values energy - ...,
...,...,...,...
575,WholeWheat_Fusilli_pasta_smokedHam,nutritional information typical values per 100...,
576,battered-chicken-balls-aldi-roosters,nutrition typical values (cooked) per 100g thi...,
577,fruit_juice2,nutritional information energy per 100ml 188kj...,
578,Lidl_Sliced_Irish_Mature_WhiteCheese,lidl nutrition per slice typical values per 10...,


In [22]:
# Define individual patterns
pattern_energy = r'(energy \d{2,}kj/\d{2,}kcal)|(energy \d{2,}kj \d{2,}kj \d{2,}kcal \d{2,}kcal)|(\d{2,}kj/\d{2,}kcal energy)|(energy\s+(\d+kj)\s+(\d+kcal))|((\d+kj)\s+(\d+kj)\s+energy\s+(\d+kcal)\s+(\d+kcal))|(\d+kj)/(\d+kcal)|(\d+ kj/\d+ kcal)|(\d+kcal)| (\d+ kcal)'
pattern_fat = r'(fat \d{1,}\.\d{1,2}g)|(fat <\d{1,}.{1,2}g)'
pattern_saturates = r'(saturates \d{1,}\.\d{1,}g)|(saturates <\d{1,}\.\d{1,}g)'
pattern_carbohydrate = r'(carbohydrate \d{1,}\.\d{1,}(g|9|0))|(carbohydrate <\d{1,}\.\d{1,}(g|9|0))'
pattern_sugars = r'(sugars \d{1,}\.\d{0,}g)|(sugars <\d{1,}\.\d{0,}g)'
pattern_fibre = r'(fibre \d{1,}\.\d{1,}g)|(fibre <\d{1,}\.\d{1,}g)'
pattern_protein = r'(protein \d{1,}\.\d{1,}g)|(protein <\d{1,}\.\d{1,}g)'
pattern_salt = r'(salt \d{1,}\.\d{1,2}(g|9))|(salt <\d{1,}\.\d{1,2}(g|9))'


# Combine the patterns with OR condition
combined_pattern = (
    pattern_energy + '|' +
    pattern_fat + '|' +
    pattern_saturates + '|' +
    pattern_carbohydrate + '|' +
    pattern_sugars + '|' +
    pattern_fibre + '|' +
    pattern_protein + '|' +
    pattern_salt
)

# Applying the combined pattern to the data
for i, row in ocr_data.iterrows():
    match = re.findall(combined_pattern, row['text'], re.MULTILINE)
    print(i,match)
    ocr_data.at[i, 'Val'] = match
    


0 [('', '', '', '', '', '', '', '', '', '', '', '', '', '', '79kcal', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''), ('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'fat 1.7g', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''), ('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'saturates 0.2g', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''), ('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'sugars 2.1g', '', '', '', '', '', '', '', '', ''), ('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'fibre 2.0g', '', '', '', '', '', '', ''), ('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'protein 1.8g', '', '', '', '', ''), ('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [89]:
#If you want to empty the csv
open('macro_nutrient_data.csv', 'w').close()

In [82]:
# Column names for nutrients
nutrient_columns = ['energy', 'protein', 'fat', 'saturates', 'carbohydrate', 'sugars', 'fibre', 'salt']

# Path to the CSV file
csv_file = 'macro_nutrient_data.csv'

# Check if the file exists and is not empty
if os.path.exists(csv_file) and os.path.getsize(csv_file) > 0:
    try:
        # Attempt to read the CSV file
        macro = pd.read_csv(csv_file)
    except pd.errors.EmptyDataError:
        # Handle the case where the file exists but is empty
        macro = pd.DataFrame(columns= ['name'] +nutrient_columns)
else:
    # Initialize an empty DataFrame if the file does not exist or is empty
    macro = pd.DataFrame(columns= ['name'] +nutrient_columns)

print(macro)

# Define the nutrient columns for the extraction
nutrient_columns_detailed = ['energy_kj', 'energy_kcal', 'fat_g', 'saturates_g', 'carbohydrate_g', 'sugars_g', 'fibre_g', 'protein_g', 'salt_g']



Empty DataFrame
Columns: [name, energy, protein, fat, saturates, carbohydrate, sugars, fibre, salt]
Index: []


In [84]:
def extract_nutrients(val_list):
    # Initialize dictionary with None values for all nutrients
    nutrients = {col: None for col in nutrient_columns}
    
    for item in val_list:  # Iterate through each tuple in the list
        for value in item:  # Iterate through each value in the tuple
            if value:  # Process non-empty values
                parts = value.split(' ')
                
                if len(parts) >= 2:
                    # There are two parts, extract nutrient and quantity
                    nutrient_name = parts[0].lower()  # Normalize to lowercase
                    quantity = ' '.join(parts[1:])  # Join the rest as quantity
                    if nutrient_name in nutrients:
                        nutrients[nutrient_name] = quantity
                        
                elif len(parts) == 1:
                    # Only quantity is present, determine if it is energy
                    quantity = parts[0]
                    if 'kcal' in quantity.lower() or 'kj' in quantity.lower():
                        # Check if 'kcal' or 'kj' is in the quantity and assign accordingly
                        if 'kcal' in quantity.lower() and nutrients['energy'] is None:
                            nutrients['energy'] = quantity
                        elif 'kj' in quantity.lower() and nutrients['energy'] is None:
                            nutrients['energy'] = quantity
    
    return nutrients


In [85]:
# List to store new rows of data
new_rows = []

# Iterate over each row in ocr_data
for index, row in ocr_data.iterrows():
    # Extract nutrients for the current row
    nutrient_values = extract_nutrients(row['Val'])
    # Add the product name to the nutrient dictionary
    nutrient_values['name'] = row['name']
    # Append the dictionary to the list of new rows
    new_rows.append(nutrient_values)

# Convert the list of dictionaries to a DataFrame
new_data = pd.DataFrame(new_rows, columns=nutrient_columns + ['name'])

# Append new data to the existing macro DataFrame
macro = pd.concat([macro, new_data], ignore_index=True)

# Remove any duplicate entries based on the 'name' column
macro.drop_duplicates(subset=['name'], inplace=True)

# Display the updated macro DataFrame
print(macro)


                                     name          energy protein    fat  \
0                         shepardless-pie          79kcal    1.8g   1.7g   
1        Lidl_TowerGate_Cookie_Cranberrry         490kcal    5.1g  22.9g   
2                  chocuer_milk_chocolate  2371kj/569kcal    7.7g  36.4g   
3          john-west-tuna-chunks-in-brine            None    None   None   
4              Lidl_PotatoWaffle_BirdsEye          94kcal    2.2g   None   
..                                    ...             ...     ...    ...   
575    WholeWheat_Fusilli_pasta_smokedHam   526kj/125kcal    9.3g   2.8g   
576  battered-chicken-balls-aldi-roosters   769kj/183kcal   16.1g   6.8g   
577                          fruit_juice2           188kj    0.5g   None   
578  Lidl_Sliced_Irish_Mature_WhiteCheese         314kcal   27.9g  22.1g   
579                                 pork3  1388kj 335kcal   18.0g  29.0g   

    saturates carbohydrate sugars fibre   salt  
0        0.2g         None   2.1g  2.0

In [86]:
macro

,name,energy,protein,fat,saturates,carbohydrate,sugars,fibre,salt
0,shepardless-pie,79kcal,1.8g,1.7g,0.2g,None,2.1g,2.0g,0.39g
1,Lidl_TowerGate_Cookie_Cranberrry,490kcal,5.1g,22.9g,10.9g,64.4g,45.2g,2.7g,0.67g
2,chocuer_milk_chocolate,2371kj/569kcal,7.7g,36.4g,None,51.6g,49.7g,None,0.18g
3,john-west-tuna-chunks-in-brine,None,None,None,None,None,None,None,None
4,Lidl_PotatoWaffle_BirdsEye,94kcal,2.2g,None,0.5g,None,0.9g,1.3g,0.58g
...,...,...,...,...,...,...,...,...,...
575,WholeWheat_Fusilli_pasta_smokedHam,526kj/125kcal,9.3g,2.8g,1.4g,None,1.6g,None,0.549
576,battered-chicken-balls-aldi-roosters,769kj/183kcal,16.1g,6.8g,0.8g,13.9g,0.5g,1.0g,0.78g
577,fruit_juice2,188kj,0.5g,None,None,None,9.5g,0.3g,None
578,Lidl_Sliced_Irish_Mature_WhiteCheese,314kcal,27.9g,22.1g,13.8g,0.80,None,None,1.80g


In [90]:
macro.to_csv('macro_nutrient_data.csv',index=False)

In [87]:
macro.isna().sum()

name              0
energy           64
protein         130
fat             140
saturates       112
carbohydrate    164
sugars          132
fibre           164
salt            108
dtype: int64